In [1]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
print('Data Loading...')
list_series_error, df_input = [], pd.read_excel('./file/1.page.xlsx',
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：1



,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,165,2


In [3]:
crawler_index, crawler_series = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                           1
 Url          https://www.dormanproducts.com/gsearch.aspx?ty...
 SKU Count                                                  165
 Page                                                         2
 Name: 0, dtype: object)

In [4]:
import time

crawler_cycle, list_url = 0, []
while len(list_url) < crawler_series['SKU Count']:
    crawler_cycle += 1

    for page in range(crawler_series['Page']):

        crawler_retry = 0
        while True:
            crawler_retry += 1
            print(crawler_cycle, page + 1, crawler_retry)
            try:
                url_request = f'''{crawler_series['Url']}&num=100&start={page * 100}'''

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                    if list_href:
                        if crawler_retry == 1 and page == 0:
                            with open('./url.html', 'w', encoding='UTF-8') as file:
                                file.write(str(soup))
                        break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = =  = = =  = = =

        list_url.extend([f'https://www.dormanproducts.com/{href.strip()}' for href in list_href])
        list_url = list(dict.fromkeys(list_url))

        # = = =  = = =  = = =

        print(f' - {len(list_url)}/{crawler_series['SKU Count']}')

        # = = =  = = =  = = =

        if len(list_url) == crawler_series['SKU Count']:
            break

list_url

1 1 1
 - 100/165
1 2 1
 - 163/165
2 1 1
2 1 2
 - 163/165
2 2 1
 - 163/165
3 1 1
 - 163/165
3 2 1
 - 163/165
4 1 1
 - 163/165
4 2 1
 - 165/165


['https://www.dormanproducts.com/p-8023-635-505.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-72452-635-513.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-129330-635-5003.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-89644-635-5002.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146177-635-157.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-102971-635-4999.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146175-635-155.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-154567-635-430.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-130016-635-555.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-24491-635-102.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-84630-635-540.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-10352-635-502.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146176-635-156.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-144781-635-144.aspx?origin=keywor

In [5]:
list_url

['https://www.dormanproducts.com/p-8023-635-505.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-72452-635-513.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-129330-635-5003.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-89644-635-5002.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146177-635-157.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-102971-635-4999.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146175-635-155.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-154567-635-430.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-130016-635-555.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-24491-635-102.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-84630-635-540.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-10352-635-502.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-146176-635-156.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-144781-635-144.aspx?origin=keywor

In [6]:
print('Data Outputting...')
df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                           'Url': list_url})
df_correct.to_excel('./test_url.xlsx', index=False)

df_correct

Data Outputting...


,No,Url
0,1,https://www.dormanproducts.com/p-8023-635-505....
1,2,https://www.dormanproducts.com/p-72452-635-513...
2,3,https://www.dormanproducts.com/p-129330-635-50...
3,4,https://www.dormanproducts.com/p-89644-635-500...
4,5,https://www.dormanproducts.com/p-146177-635-15...
...,...,...
160,161,https://www.dormanproducts.com/p-67614-904-128...
161,162,https://www.dormanproducts.com/p-67620-904-129...
162,163,https://www.dormanproducts.com/p-101363-932-98...
163,164,https://www.dormanproducts.com/p-47740-635-177...


In [7]:
print('Output Finished !')
print()

Output Finished !



In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/gsearch.aspx?ty...
 SKU Count                                                    165
 Page                                                           2
 Request_Url    https://www.dormanproducts.com/gsearch.aspx?ty...
 Name: 0, dtype: object]

In [9]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,SKU Count,Page,Request_Url
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,165,2,https://www.dormanproducts.com/gsearch.aspx?ty...


In [10]:
print('Done ~')

Done ~
